#### Simple Gen AI APP Using Langchain

In [29]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [30]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [31]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

In [32]:
loader=WebBaseLoader("https://docs.langchain.com/langsmith/billing")
loader

In [33]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/billing', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content='Manage billing in your account - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KSupportGitHubTry LangSmithTry LangSmithSearch...NavigationAccount administrationManage billing in your accountGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupReferenceOverviewPlansCreate an account and API keyAccount administrationOverviewSet up hierarchyManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesPolly (Beta)Data managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusOn this pageSet up billing for your accountDeveloper Plan: set up billing on your personal organizationPlus Plan: set up billing on a shared organizationUpdate your information (Paid pla

In [34]:
### Load Data--> Docs-->Divide our Documents into chunks documents-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [35]:
documents

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/billing', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content='Manage billing in your account - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KSupportGitHubTry LangSmithTry LangSmithSearch...NavigationAccount administrationManage billing in your accountGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupReferenceOverviewPlansCreate an account and API keyAccount administrationOverviewSet up hierarchyManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesPolly (Beta)Data managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusOn this pageSet up billing for your accountDeveloper Plan: set up billing on your personal organizationPlus Plan: set up billing on a shared organizationUpdate your information (Paid pla

In [36]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [37]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [38]:
vectorstoredb

In [39]:
## Query From a vector db
query="LangSmith has two usage limits: total traces and extended"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'For organizations with multiple workspaces only: For simplicity, LangSmith incorporates the free traces into the cost calculation of the first workspace only. In actuality, the free traces can be “consumed” by any workspace. Therefore, although workspace-level spend limits are approximate for multi-workspace organizations, the organization-level spend limit is absolute.\n\u200bConfigure trace tier distrubution\nLangSmith has two trace tiers: base traces and extended traces. Base traces have the base retention and are short-lived (14 days), while extended traces have extended retention and are long-lived (400 days). For more information, refer to the data retention conceptual docs.'

In [40]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [41]:
## Retrieval Chain, Document chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>
"""
)

prompt

ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])

In [42]:
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
| ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.comple

In [43]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph.")]
})

'LangSmith has two usage limits: one for total traces and another for extended traces. These limits correspond to the two metrics that are being tracked on the usage graph.'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [44]:
vectorstoredb

In [45]:
retriever=vectorstoredb.as_retriever()

In [47]:
from langchain_classic.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [48]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000019866E2FD90>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
            | Chat

In [49]:
response=retrieval_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})
response['answer']

"What is the retention period for LangSmith's base and extended traces?"

In [50]:
response

{'input': 'LangSmith has two usage limits: total traces and extended',
 'context': [Document(id='da824db9-a673-4eb3-a7b0-976b2e2f4101', metadata={'source': 'https://docs.langchain.com/langsmith/billing', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content='For organizations with multiple workspaces only: For simplicity, LangSmith incorporates the free traces into the cost calculation of the first workspace only. In actuality, the free traces can be “consumed” by any workspace. Therefore, although workspace-level spend limits are approximate for multi-workspace organizations, the organization-level spend limit is absolute.\n\u200bConfigure trace tier distrubution\nLangSmith has two trace tiers: base traces and extended traces. Base traces have the base retention and are short-lived (14 days), while extended traces have extended retention and are long-lived (400 days). For more information, refer to the data retention conceptual docs.'),
  Docum

In [51]:
response['context']

[Document(id='da824db9-a673-4eb3-a7b0-976b2e2f4101', metadata={'source': 'https://docs.langchain.com/langsmith/billing', 'title': 'Manage billing in your account - Docs by LangChain', 'language': 'en'}, page_content='For organizations with multiple workspaces only: For simplicity, LangSmith incorporates the free traces into the cost calculation of the first workspace only. In actuality, the free traces can be “consumed” by any workspace. Therefore, although workspace-level spend limits are approximate for multi-workspace organizations, the organization-level spend limit is absolute.\n\u200bConfigure trace tier distrubution\nLangSmith has two trace tiers: base traces and extended traces. Base traces have the base retention and are short-lived (14 days), while extended traces have extended retention and are long-lived (400 days). For more information, refer to the data retention conceptual docs.'),
 Document(id='37e5c2b6-1819-41bc-99e5-a9842823c559', metadata={'source': 'https://docs.lan